In [1]:
import numpy as np
import csv
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
#from tensorflow.examples.tutorials.mnist import input_data
from torch.autograd import Variable
import sklearn.metrics as metrics
import pickle
import os
import numpy.linalg as la
import random
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import sys
import pandas as pd
#sys.path.append("/home/CVShare/Jeni/hts/machine_learning/sampling_with_data_cleaning")
import sampling_with_data_cleaning as sdc
from utils import *

### FNN on cleaned features

In [3]:
findex = 1
x,y = get_features(findex,train=True,cleaned=True)
print y.shape
print sum(y)
y = y.astype(int)
print sum(y)
x,y = shuffle(x,y)

(47383,)
907.0
907


In [4]:
no_ex,ip_dim = x.shape
print no_ex,ip_dim

47383 154


In [5]:
class c_mlp(nn.Module):
    def __init__(self):
        super(c_mlp,self).__init__()
        self.l1 = nn.Linear(ip_dim,1000)
        self.l2 = nn.Linear(1000,1000)
        self.l3 = nn.Linear(1000,500)
        #self.l4 = nn.Linear(500,500)
        self.l5 = nn.Linear(500,100)
        self.l6 = nn.Linear(100,50)
        self.l7 = nn.Linear(50,2)
        
    def forward(self,x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.l2(x))
        x = F.leaky_relu(self.l3(x))
        #x = F.leaky_relu(self.l4(x))
        x = F.leaky_relu(self.l5(x))
        x = F.leaky_relu(self.l6(x))
        x = (self.l7(x))
        
        return x

In [6]:
max_percent = 0.76
model_no = 1
for i in range(1,6):
    val_iter = i
    print("val iter: ",val_iter)
    
   
    #weights_array = [7]
    weights_array = np.linspace(40,80,7)
    
    for i,w in enumerate(weights_array): 
        mymlp = c_mlp().cuda()
        optimizer = torch.optim.Adagrad(mymlp.parameters(),lr=1e-3)
        criterion = nn.CrossEntropyLoss(weight=torch.cuda.FloatTensor([1,w]))

        for ep in range(2500):
            
            ind = get_train_ind(val_iter=val_iter,no_examples=no_ex)
            xtrain,ytrain = get_train_batch(x,y,batch_size=100,indices=ind)
            
            model_op = mymlp(xtrain)
           
            loss = criterion(model_op,ytrain)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

    #     if(ep%30==29):
    #         print(loss.data[0])

        ## After training check on cross validation data
        xval,yval = get_val_data(x,y,no_examples=no_ex,val_iter=val_iter)
        min_fp = xval.size()[0]*0.2
        yval = yval.reshape(yval.shape[0],)
        train_op = mymlp(xval)
        train_op = train_op.cpu().data.numpy()
        pred_labels = np.argmax(train_op,axis=1)
      
        cf = metrics.confusion_matrix(yval,pred_labels).ravel()
        [tn,fp,fn,tp] = cf
        print('tn, fp, fn, tp: ',cf)
        percent = float(float(tp)/float(tp+fn))
        if(percent>max_percent*0.98):
            if(fp<min_fp):
                if(model_no<6):
                        max_percent = percent
                        print("min fp, max_percent",fp,percent)
                        model_path = os.getcwd() + '/fnn_clean_' + str(findex)+'_model'+str(model_no)
                        torch.save(mymlp.state_dict(),model_path)
                        print("saving model on val: ",val_iter," and weight: ",w)
                        model_no = model_no + 1
                else:
                    break
        


('val iter: ', 1)
('tn, fp, fn, tp: ', array([8518,  767,   45,  146]))
('min fp, max_percent', 767, 0.7643979057591623)
('saving model on val: ', 1, ' and weight: ', 40.0)
('tn, fp, fn, tp: ', array([8048, 1237,   35,  156]))
('min fp, max_percent', 1237, 0.8167539267015707)
('saving model on val: ', 1, ' and weight: ', 46.666666666666664)
('tn, fp, fn, tp: ', array([8478,  807,   40,  151]))
('tn, fp, fn, tp: ', array([8197, 1088,   40,  151]))
('tn, fp, fn, tp: ', array([7517, 1768,   26,  165]))
('min fp, max_percent', 1768, 0.8638743455497382)
('saving model on val: ', 1, ' and weight: ', 66.666666666666671)
('tn, fp, fn, tp: ', array([8221, 1064,   43,  148]))
('tn, fp, fn, tp: ', array([7504, 1781,   33,  158]))
('val iter: ', 2)
('tn, fp, fn, tp: ', array([8158, 1133,   30,  156]))
('tn, fp, fn, tp: ', array([8576,  715,   61,  125]))
('tn, fp, fn, tp: ', array([8245, 1046,   37,  149]))
('tn, fp, fn, tp: ', array([8217, 1074,   32,  154]))
('tn, fp, fn, tp: ', array([8174, 111

KeyboardInterrupt: 

In [7]:
xtest_t,ytest = get_features(findex,train=False)
xtest = Variable(torch.cuda.FloatTensor(xtest_t).cuda())
final_pred = np.zeros(ytest.shape[0],)
print final_pred.shape
for i in range(1,6):
    model_path = os.getcwd() + '/fnn_clean_' + str(findex)+'_model'+str(i)
    
    
    test_model = c_mlp().cuda()
    test_model.load_state_dict(torch.load(model_path))
    test_op = test_model(xtest)
    test_op = test_op.cpu().data.numpy()
    pred_labels = np.argmax(test_op,axis=1).reshape(final_pred.shape[0],)
    print pred_labels.shape
    final_pred = pred_labels + final_pred
    print final_pred.shape
    cf = metrics.confusion_matrix(ytest,pred_labels).ravel()
    print('tn, fp, fn, tp: ',cf)

(11957,)
(11957,)
(11957,)
('tn, fp, fn, tp: ', array([10868,  1047,    22,    20]))
(11957,)
(11957,)
('tn, fp, fn, tp: ', array([10207,  1708,    15,    27]))
(11957,)
(11957,)
('tn, fp, fn, tp: ', array([9555, 2360,   11,   31]))
(11957,)
(11957,)
('tn, fp, fn, tp: ', array([9632, 2283,   12,   30]))
(11957,)
(11957,)
('tn, fp, fn, tp: ', array([9578, 2337,   14,   28]))


In [8]:
final_pred[final_pred>1] = 1
cf = metrics.confusion_matrix(ytest,final_pred).ravel()
print('tn, fp, fn, tp: ',cf)

('tn, fp, fn, tp: ', array([8561, 3354,    8,   34]))


In [ ]:
pred_labels.shape

In [ ]:
final_pred.shape